In [2]:
import torch

In [4]:
a = torch.arange(4).reshape(2, 2)
b = torch.arange(6).reshape(3, 2)
print(a)
print(b)

tensor([[0, 1],
        [2, 3]])
tensor([[0, 1],
        [2, 3],
        [4, 5]])


### 广播机制
正常来说,行列不同的矩阵不能相加,但 pytorch 可以任意相同维度的矩阵相加

In [5]:
r1 = a + b
print(r1)

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 0

#### 理解错误
pytorch 实现不同行列矩阵相加参考了矩阵乘法,那么就需要前一个矩阵的列 = 后一个矩阵的行

In [7]:
b = b.reshape(2, 3)
print(b)

tensor([[0, 1, 2],
        [3, 4, 5]])


In [8]:
r1 = a + b
print(r1)

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

#### 再次理解错误
似乎只支持<strong>一维</strong>不同行列矩阵相加

In [9]:
a = torch.arange(4).reshape(4, 1)
b = torch.arange(3).reshape(1, 3)
print(a, b)
r = a + b
print(r)

tensor([[0],
        [1],
        [2],
        [3]]) tensor([[0, 1, 2]])
tensor([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]])


#### view 和 reshape
PyTorch 中的 view() 和 reshape() 函数都可以用来重新调整张量的形状。它们的效果是一样的，但是有一些区别12。  
- view() 方法只能改变连续的 (contiguous) 张量，否则需要先调用 .contiguous() 方法；而 .reshape() 方法不受此限制；如果对 tensor 调用过 transpose, permute 等操作的话会使该 tensor 在内存中变得不再连续。
- .view() 方法返回的张量总是和原来的张量共享一份相同的数据，而 .reshape() 在新形状满足一定条件时会共享相同一份数据，否则会复制一份新的数据。
- 两者对于原始张量的连续性要求不同。

In [1]:
a = torch.arange(1 << 3)
print(f"source a array: {a}")
c = a.view(1 << 2, 2)
c[:] = 2
print(f"after view: {a}")
b = a.reshape(1 << 2, 2)
b[:] = 2
print(f"after reshape: {a}")

NameError: name 'torch' is not defined

In [3]:
a = torch.arange(1 << 3).reshape(2, 2, 2)
a

tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])

In [6]:
print(a.sum(axis=0))
print(a.sum(axis=1))
# 第二个维度求和就是 将第二个维度里面的矩阵进行相加以去除第二个维度 例如 0 + 2, 1 + 3

tensor([[ 4,  6],
        [ 8, 10]])
tensor([[ 2,  4],
        [10, 12]])


我觉得是分子布局，X和W内积是标量，<X,W>(标量)对W(列向量)求偏导=X转置(行向量)   符合分子布局